1. 문서 내용을 읽는다.
2. 문서를 쪼갠다.
    - 토큰수 초과로 답변을 생성하지 못할 수 있고
    - 문서가 길면 답변 생성이 오래걸림
3. 임베딩 -> 벡터 데이터베이스에 저장
4. 질문이 있을 때, 벡터 데이터베이스에 유사도 검색
5. 유사도 검색으로 가져온 문서를 LLM에 질문과 같이 전달

In [54]:
from langchain.chains import RetrievalQA
from dotenv import load_dotenv
from langchain_openai import OpenAIEmbeddings
from pinecone import Pinecone
from langchain_pinecone import PineconeVectorStore
from langchain_core.output_parsers import StrOutputParser
from langchain_core.prompts import ChatPromptTemplate

load_dotenv()

embedding = OpenAIEmbeddings(model='text-embedding-3-large')
index_name = 'tax-markdown-index'
database = PineconeVectorStore.from_documents(document_list, embedding, index_name=index_name)
retriever = database.as_retriever(search_kwargs={'k': 4})

qa_chain = RetrievalQA.from_chain_type(
    llm, retriever = retriever,
    chain_type_kwargs = {"prompt": prompt}
)

query =  '연봉 5천만 원인 거주자의 소득세는 얼마인가요?'
# query -> 직장인 -> 거주자 변경하는 chain 추가
ai_message = qa_chain({"query": query})

dictionary = ["사람을 나타내는 표현 -> 거주자 "]

prompt = ChatPromptTemplate.from_template(f"""
    사용자의 질문을 보고, 우리의 사전을 참고해서 사용자의 질문을 변경해주세요.
    만약 변경할 필요가 없다고 판단된다면, 사용자의 질문을 변경하지 않은 채로 질문만 리턴해주세요.
    사전: {dictionary}
    
    질문: {{question}}
""")

dictionary_chain = prompt | llm | StrOutputParser()
tax_chain = {"query":dictionary_chain} | qa_chain
ai_response = tax_chain.invoke({"question": query})